In [1]:
# Cell 1: Kiểm tra môi trường
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

import torch
print("✅ PyTorch version:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())

Python executable: /root/miniconda3/envs/chest-xray/bin/python
Python version: 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
✅ PyTorch version: 2.8.0+cu129
✅ CUDA available: True
